In [1]:
from prody import *
from pylab import *
import numpy as np
from os.path import basename
import fnmatch
import os
import re
import itertools
import shutil

### Negative to positive, annihilate two potassium ions

In [2]:
wt_aa = "ASP"
position_aa = 69
neg_char_aa = ["ASP", "GLU"]
pos_char_aa = ["ARG", "LYS"]
neu_char_aa = ["ALA","ASN","CYS","GLN","HSD","LEU","MET","PHE","SER","THR","TRP","TYR","VAL"]
#mutation_list = ["ALA","ARG","ASN","ASP","CYS","GLU","GLN","GLY","HIS","ILE", "LEU","LYS","MET","PHE","PRO","SER","THR","TRP","TYR","VAL"]

In [3]:
file_pattern = re.compile("wt_ap_100_chainA_"+wt_aa+"_"+str(position_aa)+"_("+"|".join(pos_char_aa)+")_fep.fep$")
for file in os.listdir('.'):
    if re.search(file_pattern, file):
        pdb = file
        print(file)
        file_name_wh_ex_ = str(os.path.splitext(pdb)[0])
        #os.rename(str(file_name_wh_ex_+".fep"), str(file_name_wh_ex_+".pdb"))
        shutil.copyfile(str(file_name_wh_ex_+".fep"), str(file_name_wh_ex_+".pdb"))
        file_name_wh_ex = str(os.path.splitext(pdb)[0]).replace("_chainA","")
        resid = pdb.split('_')[5]
        structure_1 = parsePDB(str(file_name_wh_ex_+".pdb"))
        structure_2 = parsePDB(str(file_name_wh_ex+"_wb_ionized.pdb"))
        print(structure_1)
        betas = structure_1.select("chain P and resnum "+str(resid))
        print(betas)
        betas = betas.getBetas()
        selection_str_2 = structure_2.select("chain P and resnum "+str(resid))
        selection_str_2_betas = selection_str_2.setBetas(betas)
        ions = structure_2.select("resname POT")
        ion_resnums = ions.getResnums()
        ions_anni = []
        for at in itertools.combinations(range(len(ions)), 2):
            coor_ion1 = ions[at[0]].getCoords()
            coor_ion2 = ions[at[1]].getCoords()
            mut_site = structure_2.select("resnum "+str(resid)+" and name CA").getCoords()
            distum_1 = calcDistance(coor_ion1, coor_ion2)
            distum_2 = calcDistance(coor_ion1, mut_site)
            distum_3 = calcDistance(coor_ion2, mut_site)
            if (20 < distum_1 < 35) and (20 < distum_2 < 35) and (20 < distum_3 < 35):
                ions_anni.extend([ion_resnums[at[0]], ion_resnums[at[1]]])
                print(ions_anni)
                break
        ion_sele = str(' '.join(map(str,ions_anni)))
        print(ion_sele)
        selection_str_3 = structure_2.select("resname POT and resnum "+ion_sele)
        selection_str_3_betas = selection_str_3.setBetas(-1)
        
        writePDB(str(file_name_wh_ex+"_wb_ionized.fep"), structure_2, autoext=False)
        os.rename(str(file_name_wh_ex+"_wb_ionized.fep.pdb"), str(file_name_wh_ex+"_wb_ionized.fep"))

@> 2505 atoms and 1 coordinate set(s) were parsed in 0.04s.


wt_ap_100_chainA_ASP_69_LYS_fep.fep


@> 23091 atoms and 1 coordinate set(s) were parsed in 0.38s.


AtomGroup wt_ap_100_chainA_ASP_69_LYS_fep
Selection 'chain P and resnum 69'
[4, 16]
4 16


@> 2507 atoms and 1 coordinate set(s) were parsed in 0.04s.


wt_ap_100_chainA_ASP_69_ARG_fep.fep


@> 23093 atoms and 1 coordinate set(s) were parsed in 0.37s.


AtomGroup wt_ap_100_chainA_ASP_69_ARG_fep
Selection 'chain P and resnum 69'
[5, 7]
5 7


In [4]:
structure_1.select("protein and resnum 69")

In [5]:
structure_1.select("resnum 69")

<Selection: 'resnum 69' from wt_ap_100_chainA_ASP_69_ARG_fep (30 atoms)>

In [6]:
print("protein and resnum "+str(resid))

protein and resnum 69


### Negative to neutral, annihilate one potassium ion

In [7]:
file_pattern = re.compile("wt_ap_100_chainA_"+wt_aa+"_"+str(position_aa)+"_("+"|".join(neu_char_aa)+")_fep.fep$")
for file in os.listdir('.'):
    if re.search(file_pattern, file):
        pdb = file
        print(file)
        file_name_wh_ex_ = str(os.path.splitext(pdb)[0])
        #os.rename(str(file_name_wh_ex_+".fep"), str(file_name_wh_ex_+".pdb"))
        shutil.copyfile(str(file_name_wh_ex_+".fep"), str(file_name_wh_ex_+".pdb"))
        file_name_wh_ex = str(os.path.splitext(pdb)[0]).replace("_chainA","")
        resid = pdb.split('_')[5]
        structure_1 = parsePDB(str(file_name_wh_ex_+".pdb"))
        structure_2 = parsePDB(str(file_name_wh_ex+"_wb_ionized.pdb"))
        betas = structure_1.select("chain P and resnum "+str(resid))
        betas = betas.getBetas()
        selection_str_2 = structure_2.select("chain P and resnum "+str(resid))
        selection_str_2_betas = selection_str_2.setBetas(betas)
        ions = structure_2.select("resname POT")
        ion_resnums = ions.getResnums()
        for at in range(len(ions)):
            coor_ion1 = ions[at].getCoords()
            mut_site = structure_2.select("resnum "+str(resid)+" and name CA").getCoords()
            distum_1 = calcDistance(coor_ion1, mut_site)
            if 20 < distum_1 < 35:
                ions_anni = ion_resnums[at]
                break
        selection_str_3 = structure_2.select("resname POT and resnum "+str(ions_anni))
        selection_str_3_betas = selection_str_3.setBetas(-1)
        
        writePDB(str(file_name_wh_ex+"_wb_ionized.fep"), structure_2, autoext=False)
        os.rename(str(file_name_wh_ex+"_wb_ionized.fep.pdb"), str(file_name_wh_ex+"_wb_ionized.fep"))

@> 2500 atoms and 1 coordinate set(s) were parsed in 0.04s.


wt_ap_100_chainA_ASP_69_GLN_fep.fep


@> 23084 atoms and 1 coordinate set(s) were parsed in 0.37s.
@> 2504 atoms and 1 coordinate set(s) were parsed in 0.04s.


wt_ap_100_chainA_ASP_69_TYR_fep.fep


@> 23088 atoms and 1 coordinate set(s) were parsed in 0.35s.
@> 2503 atoms and 1 coordinate set(s) were parsed in 0.04s.


...

@> 23078 atoms and 1 coordinate set(s) were parsed in 0.35s.
@> 2500 atoms and 1 coordinate set(s) were parsed in 0.04s.


wt_ap_100_chainA_ASP_69_HSD_fep.fep


@> 23084 atoms and 1 coordinate set(s) were parsed in 0.35s.
@> 2497 atoms and 1 coordinate set(s) were parsed in 0.04s.


wt_ap_100_chainA_ASP_69_THR_fep.fep


@> 23081 atoms and 1 coordinate set(s) were parsed in 0.35s.


### Negative to negative, no ion annihilation

In [8]:
file_pattern = re.compile("wt_ap_100_chainA_"+wt_aa+"_"+str(position_aa)+"_("+"|".join(neg_char_aa)+")_fep.fep$")
for file in os.listdir('.'):
    if re.search(file_pattern, file):
        pdb = file
        print(file)
        file_name_wh_ex_ = str(os.path.splitext(pdb)[0])
        #os.rename(str(file_name_wh_ex_+".fep"), str(file_name_wh_ex_+".pdb"))
        shutil.copyfile(str(file_name_wh_ex_+".fep"), str(file_name_wh_ex_+".pdb"))
        file_name_wh_ex = str(os.path.splitext(pdb)[0]).replace("_chainA","")
        resid = pdb.split('_')[5]
        structure_1 = parsePDB(str(file_name_wh_ex_+".pdb"))
        structure_2 = parsePDB(str(file_name_wh_ex+"_wb_ionized.pdb"))
        betas = structure_1.select("chain P and resnum "+str(resid))
        betas = betas.getBetas()
        selection_str_2 = structure_2.select("chain P and resnum "+str(resid))
        selection_str_2_betas = selection_str_2.setBetas(betas)
        writePDB(str(file_name_wh_ex+"_wb_ionized.fep"), structure_2, autoext=False)
        os.rename(str(file_name_wh_ex+"_wb_ionized.fep.pdb"), str(file_name_wh_ex+"_wb_ionized.fep"))

@> 2498 atoms and 1 coordinate set(s) were parsed in 0.04s.


wt_ap_100_chainA_ASP_69_GLU_fep.fep


@> 23080 atoms and 1 coordinate set(s) were parsed in 0.35s.
